In [1]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from torch.utils.data import random_split
from Utilities.dataset import EEGDataset, Compose
from Utilities.preprocessing import EEGDataProcessor

In [2]:
pp = EEGDataProcessor()

pp.DOWNSAMPLED_FREQ = 1024

transforms = Compose([
    pp.correct_offset,
    pp.filter,
    pp.downsample,
    pp.normalize,
])

dataset = EEGDataset("./dataset", transforms)

train_set, test_set, validation_set = random_split(dataset, [0.7, 0.2, 0.1])

train_loader = DataLoader(train_set, batch_size=32, num_workers=4, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, num_workers=4)
val_loader = DataLoader(validation_set, batch_size=32, num_workers=4)

In [3]:
for data, label in train_loader:
    print(data.shape)
    break

torch.Size([32, 16, 4096])


In [10]:
import torchmetrics
from torch.optim import Adam
from torch.nn.functional import binary_cross_entropy, one_hot
import lightning.pytorch as pl

class SignalCNN(pl.LightningModule):
    channels = 16
    
    def __init__(self):
        super(SignalCNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv1d(
                in_channels = self.channels, 
                out_channels = 32, 
                kernel_size = 11,
                stride = 3,
                dilation = 3
            ),
            nn.ReLU(),
            nn.AvgPool1d(2),
            nn.BatchNorm1d(32),
            nn.Dropout1d(p=0.6),
            
            nn.Conv1d(
                in_channels = 32, 
                out_channels = 64, 
                kernel_size = 5,
                stride = 2,
                dilation = 1
            ),
            nn.ReLU(),
            nn.AvgPool1d(2),
            nn.BatchNorm1d(64),
            nn.Dropout1d(p=0.6),
            
            nn.Conv1d(
                in_channels = 64, 
                out_channels = 128, 
                kernel_size = 3,
                stride = 2,
                dilation = 1
            ),
            nn.ReLU(),
            nn.AvgPool1d(3),
            nn.BatchNorm1d(128),
            nn.Dropout1d(p=0.6),
            
            nn.Conv1d(
                in_channels = 128, 
                out_channels = 128, 
                kernel_size = 1,
                stride = 2,
                dilation = 1
            ),
            nn.ReLU(),
            nn.AvgPool1d(3),
            nn.BatchNorm1d(128),
            nn.Dropout1d(p=0.6),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(1152, 512),
            nn.BatchNorm1d(512),
            nn.Dropout1d(p=0.6),
            
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.Dropout1d(p=0.6),
            
            nn.Linear(128, 3),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x
    
    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=1e-4)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        data, label = batch
        
        # get predictions
        output = self(data)
        
        # convert for loss calculation
        output = torch.sigmoid(output)
        label = one_hot(label,num_classes=3).to(torch.float32)
    
        # calculate loss
        loss = binary_cross_entropy(output, label)
        self.log("training_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        data, label = batch
        
        # get predictions
        output = self(data)
        
        # convert for loss calculation
        output = torch.sigmoid(output)
        label = one_hot(label,num_classes=3).to(torch.float32)
    
        # calculate loss
        loss = binary_cross_entropy(output, label)
        self.log("test_loss", loss)
    
    def validation_step(self, batch, batch_idx):
        data, label = batch
        
        # get predictions
        output = self(data)
        
        # convert for loss calculation
        output = torch.sigmoid(output)
        label = one_hot(label,num_classes=3).to(torch.float32)
    
        # calculate loss
        loss = binary_cross_entropy(output, label)
        self.log("val_loss", loss)

In [11]:
model = SignalCNN()

# Train

In [12]:
trainer = pl.Trainer(max_epochs=20)
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type        | Params
--------------------------------------------
0  | conv1      | Conv2d      | 1.0 K 
1  | batchnorm1 | BatchNorm2d | 32    
2  | padding1   | ZeroPad2d   | 0     
3  | conv2      | Conv2d      | 260   
4  | batchnorm2 | BatchNorm2d | 8     
5  | pooling2   | MaxPool2d   | 0     
6  | padding2   | ZeroPad2d   | 0     
7  | conv3      | Conv2d      | 516   
8  | batchnorm3 | BatchNorm2d | 8     
9  | pooling3   | MaxPool2d   | 0     
10 | fc1        | Linear      | 57    
--------------------------------------------
1.9 K     Trainable params
0         Non-trainable params
1.9 K     Total params
0.008     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [16, 1, 1, 64], expected input[1, 24, 16, 4096] to have 1 channels, but got 24 channels instead